# Laboratory Scheduling Using Constraint Satisfaction Programming (CSP)

**Group Members:**
- Aime Muganga - 670232
- Racheal Momita - 665601
- Zakariya Shafi - 668596
- Sean Nderitu - 669648
- Crispin Ndungu - 669837


## Concept of Constraint Satisfaction Problems (CSP)

CSPs (Constraint Satisfaction Problems) are search problems where the solution must satisfy a set of constraints. The problem is framed as a collection of variables, each with a set of possible values (domains), and a collection of constraints that restrict the values the variables can take simultaneously.

This approach is powerful in situations where we must make optimal or feasible decisions given limited resources or rules — like scheduling, assignment, or planning problems.

### Components of CSP:

- **Variables:** These are the unknowns we are solving for. In a scheduling problem, they can represent elements like time slots, locations, or resources.

- **Domains:** Each variable has a domain — a set of values it can take. For example, a time slot variable might have a domain of 7 possible time blocks in a day.

- **Constraints:** These are the rules that must be respected. They ensure the feasibility of the final assignment. Constraints can be:
  - **Hard constraints**, which must be strictly satisfied (e.g., no faculty can teach two classes at the same time).
  - **Soft constraints**, which are desirable but not mandatory.

A CSP solver uses these constraints, along with heuristics, to reduce the search space and find a feasible solution efficiently.


## Problem Statement: Laboratory Scheduling at UISU-A

We aim to design a weekly laboratory schedule for **5 laboratories** in the LKB Building at **UISU-Africa**. Our university offers **200 classes per semester**, taught by **60 faculty members**.

To automate this process, we adopted a **Constraint Satisfaction Programming (CSP)** approach, implemented using **Google OR-Tools**, a robust library for constraint solving.

Initially, we considered using the `constraint` Python library, but we found it ineffective for scaling and handling the complexity of the problem. Hence, we transitioned to using `ortools.sat.python.cp_model`, which proved more efficient and flexible.

### Input Format

Our program takes input from an Excel file named `classes_input2.xlsx` with the following columns:

- `id`: Unique class identifier (e.g., DSA2020)
- `faculty`: ID of the faculty member assigned to teach the class (0–59)
- `is_concentration`: Whether the class is a concentration course (1 for yes, 0 for no)
- `is_double`: Whether the class is a double session class (1 for yes, 0 for no)

The program returns an output Excel file named `labo_schedule3.xlsx` with the complete lab schedule.

### Variables, Domain, and Constraints

#### Variables:
- **Lab** assigned to each class
- **Time slot** for each class
- **Day pair** (e.g., Mon/Wed, Tue/Thu, or single days)

#### Domains:
- **Lab:** Lab1 to Lab5
- **Time slot:** 7 possible slots in a day
- **Day pair:** 7 combinations of days (MonWed, TueThu, Mon, Tue, etc.)

#### Constraints:
1. **No time overlap for same-semester classes**
2. **No faculty teaching more than one class at the same time**
3. **Concentration classes must be held in smaller labs (Lab4 or Lab5)**
4. **Double classes must be scheduled on Mon/Wed or Tue/Thu**

> Note: While the university offers 200 classes, we reduced the number to **100** in our experiment due to performance limitations on our local machines.


## Implementation Overview

We now walk through the implementation of our CSP model using OR-Tools.

The code follows these major steps:

1. **Reading and validating input data**
2. **Creating decision variables for lab, timeslot, and daypair**
3. **Defining hard constraints**
4. **Adding special rules for concentration and double classes**
5. **Solving the model using CP-SAT solver**
6. **Exporting the resulting schedule to Excel**

## GUI option
To make our scheduling tool more accessible and user-friendly, we also developed a simple web application using Streamlit, which provides an interactive interface for uploading the course input file and downloading the generated schedule. The web app is implemented in the file named GUI.py. To run it, make sure the streamlit library is installed (pip install streamlit), and then execute the command streamlit run main2.py in your terminal. This will launch the app in your web browser, allowing you to use the scheduling system without interacting directly with the code.




### Step 1: Defining Variables and Domains(Assigned to Aime)

We create three variables for each class:
- `lab`: An integer between 0 and 4, representing the 5 labs.
- `slot`: An integer between 0 and 6, representing the 7 time slots per day.
- `daypair`: An integer between 0 and 6, representing the 7 day combinations.

These are defined using `model.NewIntVar()`:

```python
lab = model.NewIntVar(0, len(labs_all) - 1, f"{cls['id']}_lab")
slot = model.NewIntVar(0, len(timeslots) - 1, f"{cls['id']}_slot")
daypair = model.NewIntVar(0, len(day_pairs) - 1, f"{cls['id']}_daypair")
```

The variables are stored in a dictionary for easy access per class.


### Step 2: Defining Constraints(Assigned to Zakariya and Rachael)

#### A. Avoiding Lab-Time-Day Overlaps
No two classes should be in the same lab at the same time on the same day. This is ensured by:

```python
model.AddBoolOr([same_lab.Not(), same_slot.Not(), same_day.Not()])
```

#### B. Faculty Conflict Avoidance
No faculty member can teach two classes at the same time (same slot and same day):

```python
if cls1["faculty"] == cls2["faculty"]:
    model.Add(same_time == 0)
```

#### C. Concentration Classes in Small Labs
Classes marked as concentration must be scheduled only in Lab4 or Lab5:

```python
model.AddAllowedAssignments([lab], [(3,), (4,)])
```

#### D. Double Classes on Mon/Wed or Tue/Thu Only
These are restricted to specific day pairs:

```python
allowed_double_days = [day_pairs.index("MonWed"), day_pairs.index("TueThu")]
```

All these constraints ensure the solution is practical, realistic, and follows institutional rules.


### Step 3: Solving the Model and Exporting the Output(Assigned to Sean and Crispin)

We solve the problem using:

```python
solver = cp_model.CpSolver()
status = solver.Solve(model)
```

If a feasible or optimal solution is found, we extract the values and format them into a readable DataFrame. The final schedule is saved as an Excel file:

```python
df.to_excel("labo_schedule3.xlsx", index=False)
```

This file contains:
- Class ID
- Faculty ID
- Assigned Lab
- Time Slot
- Days
- Type of Class (Concentration or Regular)
- Whether it’s a Double Class


## Conclusion

This project demonstrates the power of CSPs in solving real-world scheduling problems. Using OR-Tools, we modeled variables, defined their domains, and encoded institutional constraints to automatically generate a lab schedule that avoids conflicts and follows policy rules.

Despite initial performance limitations with other libraries and with large input sizes, our final approach is scalable and can be further optimized or extended to include soft constraints (like preferred times or faculty availability) in future versions.

This lab scheduling system can greatly ease administrative workload and improve scheduling efficiency at UISU-A.


## Code Part

In [5]:
from ortools.sat.python import cp_model
import pandas as pd
import random

# Setup parameters
num_classes = 100  # Adjust as needed (e.g., 200)
num_faculty = 60
labs_all = ['Lab1', 'Lab2', 'Lab3', 'Lab4', 'Lab5']
labs_small = ['Lab4', 'Lab5']
timeslots = ["7-8:40", "9-10:40", "11-12:40", "1:20-3", "3:30-5:10", "5:30-7:10", "7:30-9:10"]
day_pairs = ["MonWed", "TueThu", "Mon", "Tue", "Wed", "Thu", "Fri"]

# Generate dummy class data
# Generate dummy class data with course codes ending in 0
df_input = pd.read_excel("classes_input2.xlsx")

# Validate expected columns exist
expected_cols = {"id", "faculty", "is_concentration", "is_double"}
if not expected_cols.issubset(df_input.columns):
    raise ValueError(f"Input Excel file must contain columns: {expected_cols}")

# Convert dataframe rows into list of dictionaries
classes = df_input.to_dict(orient="records")

model = cp_model.CpModel()

# Create variables
class_vars = {}
for cls in classes:
    lab = model.NewIntVar(0, len(labs_all) - 1, f"{cls['id']}_lab")
    slot = model.NewIntVar(0, len(timeslots) - 1, f"{cls['id']}_slot")
    daypair = model.NewIntVar(0, len(day_pairs) - 1, f"{cls['id']}_daypair")
    class_vars[cls["id"]] = {"lab": lab, "slot": slot, "daypair": daypair}

# Add constraints
for i, cls1 in enumerate(classes):
    v1 = class_vars[cls1["id"]]
    for j in range(i + 1, len(classes)):
        cls2 = classes[j]
        v2 = class_vars[cls2["id"]]

        # Boolean variables for comparisons
        same_lab = model.NewBoolVar(f"same_lab_{i}_{j}")
        model.Add(v1["lab"] == v2["lab"]).OnlyEnforceIf(same_lab)
        model.Add(v1["lab"] != v2["lab"]).OnlyEnforceIf(same_lab.Not())

        same_slot = model.NewBoolVar(f"same_slot_{i}_{j}")
        model.Add(v1["slot"] == v2["slot"]).OnlyEnforceIf(same_slot)
        model.Add(v1["slot"] != v2["slot"]).OnlyEnforceIf(same_slot.Not())

        same_day = model.NewBoolVar(f"same_day_{i}_{j}")
        model.Add(v1["daypair"] == v2["daypair"]).OnlyEnforceIf(same_day)
        model.Add(v1["daypair"] != v2["daypair"]).OnlyEnforceIf(same_day.Not())

        # Forbid same lab AND same slot AND same day simultaneously
        model.AddBoolOr([same_lab.Not(), same_slot.Not(), same_day.Not()])

        # Faculty conflict: same faculty cannot teach two classes at the same time
        if cls1["faculty"] == cls2["faculty"]:
            # same time means same slot and same day
            same_time = model.NewBoolVar(f"same_time_{i}_{j}")
            model.AddBoolAnd([same_slot, same_day]).OnlyEnforceIf(same_time)
            model.AddBoolOr([same_slot.Not(), same_day.Not()]).OnlyEnforceIf(same_time.Not())
            model.Add(same_time == 0)  # no overlap for same faculty

# Concentration classes must be scheduled in small labs
for cls in classes:
    if cls["is_concentration"]:
        model.AddAllowedAssignments(
            [class_vars[cls["id"]]["lab"]],
            [(labs_all.index(lab),) for lab in labs_small]
        )

# Double classes only on MonWed or TueThu
allowed_double_days = [day_pairs.index("MonWed"), day_pairs.index("TueThu")]
for cls in classes:
    if cls["is_double"]:
        model.AddAllowedAssignments(
            [class_vars[cls["id"]]["daypair"]],
            [(d,) for d in allowed_double_days]
        )

# Solve model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output result
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    schedule = []
    for cls in classes:
        v = class_vars[cls["id"]]
        schedule.append({
            "Class": cls["id"],
            "Faculty": f"Faculty_{cls['faculty'] + 1}",
            "Lab": labs_all[solver.Value(v["lab"])],
            "Time": timeslots[solver.Value(v["slot"])],
            "Days": day_pairs[solver.Value(v["daypair"])],
            "Type": "Concentration" if cls["is_concentration"] else "Regular",
            "Double Class": "Yes" if cls["is_double"] else "No"
        })
    df = pd.DataFrame(schedule)
    print(df.to_string(index=False))
    df.to_excel("lab_schedule.xlsx", index=False)

else:
    print("No feasible schedule found.")

  Class    Faculty  Lab      Time   Days          Type Double Class
SWE2160 Faculty_13 Lab5   9-10:40    Fri Concentration           No
APT1510 Faculty_56 Lab3  11-12:40 MonWed       Regular          Yes
IST2010 Faculty_43 Lab2 3:30-5:10 MonWed       Regular          Yes
DSA2990 Faculty_20 Lab4 5:30-7:10 MonWed Concentration          Yes
DSA1750 Faculty_29 Lab5 5:30-7:10 MonWed Concentration          Yes
DSA3330 Faculty_32 Lab1 5:30-7:10 TueThu       Regular          Yes
SWE3690  Faculty_7 Lab1 5:30-7:10 MonWed       Regular           No
DSA2080 Faculty_25 Lab4 5:30-7:10    Tue Concentration           No
IST1020 Faculty_25 Lab3    1:20-3 MonWed       Regular          Yes
DSA3690 Faculty_35 Lab2   9-10:40 TueThu       Regular          Yes
APT1900 Faculty_30 Lab3 7:30-9:10 MonWed       Regular           No
DSA1920 Faculty_55 Lab2    7-8:40 TueThu       Regular           No
SWE3410 Faculty_50 Lab3 3:30-5:10    Wed       Regular           No
SWE2680 Faculty_29 Lab1    7-8:40    Tue       R